## Instalación de plastimatch en Colab

Google Colab utiliza una distribución Ubuntu sin paquete `plastimatch`. Si intentamos `apt install plastimatch` nos devuelve `Package 'plastimatch' has no installation candidate`.

Afortunadamente el módulo `pyplastimatch` resuelve esta limitación empequetando su propia compilación.

La siguiente celda instala el ejecutable `plastimatch` y el módulo `pyplastimatch` que lo encapsula.

In [ ]:
!pip -q install pyplastimatch
from pyplastimatch.utils.install import install_precompiled_binaries
install_precompiled_binaries()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 9.5 MB/s eta 0:00:00
PyPlastimatch Plastimatch installation utility.
NOTE: this utility is not meant to be replace the normal install of Plastimatch via apt.
Rather, it is meant to be used in case a Plastimatch binary is not available for a specific distribution.

System distribution: Ubuntu 22.04

Matching distribution found in the latest PyPlastimatch release.


Installing binaries... Done.

Creamos un directorio para manejar los datos sintéticos

In [2]:
!mkdir -p mydata/cuba/ss

Corremos una secuencia de comandos de plastimatch
- Creación de las imágenes del paciente, una cuba de agua
- Dos máscaras para la estructuras:
  + BODY: coincide con el contorno externo de la cuba
  + PTV: Una esfera en el centro de la cuba
- Creamos el archivo DICOM de estructuras a partir de las máscaras anteriores

In [ ]:
# Creación de la estructura DICOM de datos. Plastimatch crea los archivos de imagen, un archivo de estructuras y otro de dosis.
!plastimatch synth \
  --background -1000 \
  --foreground 0 \
  --pattern rect \
  --output-dicom "mydata/cuba" \
  --rect-size "-50 50 -50 50 -50 50" \
  --origin "-55 -55 -55" \
  --spacing "1 1 1" \
  --dim "110 110 110" \
  --patient-id "TestDICOM01" \
  --patient-name "CUBA^MANIQUI" \
  --patient-pos "hfs"

# plastimatch ha creado un archivo de estructurs y uno de dosis que no queremos
!rm -f mydata/cuba/rtss*.dcm
!rm -f mydata/cuba/dose*.dcm

# Creación de la máscara para el contorno externo
!plastimatch synth \
  --background 0 \
  --foreground 1 \
  --pattern rect \
  --output "mydata/cuba/ss/BODY.mha" \
  --rect-size "-50 50 -50 50 -50 50" \
  --origin "-55 -55 -55" \
  --spacing "1 1 1" \
  --dim "500 500 500"

# Creación de la máscara para el PTV
!plastimatch synth \
  --background 0 \
  --foreground 1 \
  --pattern "sphere" \
  --output "mydata/cuba/ss/PTV.mha" \
  --radius "10" \
  --center "0 0 0" \
  --origin "-55 -55 -55" \
  --spacing "1 1 1" \
  --dim "500 500 500"

# Creación de un archivo DICOM de estructuras a partir de las máscaras manteniendo los demográficos del paciente.
!plastimatch convert \
  --input-prefix mydata/cuba/ss \
  --output-dicom mydata/cuba \
  --referenced-ct mydata/cuba

Bajamos los archivos de la nube Colab a nuestro disco local.
- Importación del módulo necesario
- Comprimimos los datos para facilitar la descarga
. Descargamos
- Borramos los archivos

In [6]:
from google.colab import files

In [7]:
!zip -q -j /content/cuba.zip /content/mydata/cuba/*

In [8]:
files.download('/content/cuba.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
!rm -f /content/cuba.zip

In [10]:
!rm -rf mydata